In [58]:
# I discussed with B09901133 and B09502019. They gave me some hints that the testing data may have some problems (WS_HR and PM2.5 values are strange)

import numpy as np
import csv
import math
import pandas as pd
import os
import random

os.getcwd()
seed = 9487
random.seed(seed)
!gdown --id "1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy" --output "train.csv"
!gdown --id '1FXJztppYG9Q4b_4NHvcmPsc4o5obceWC' --output "test.csv"



# Incase the links above die, you can use the following instead.
#!gdown --id '11abE854Eyv4BA7qt5k8r_80sJ3KuOQUN' --output "train.csv"
#!gdown --id '1uod-Z4ztluXnuHtgUbm39nMudUKqXHMl' --output "test.csv"

# If the data is still missing, you can manually download it from kaggle, and upload the files under /content


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy
To: /content/train.csv
100% 324k/324k [00:00<00:00, 87.9MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1FXJztppYG9Q4b_4NHvcmPsc4o5obceWC
To: /content/test.csv
100% 49.0k/49.0k [00:00<00:00, 52.8MB/s]


In [59]:
def mean(a):
  total = 0
  for i in range (len(a)):
    total += a[i]
  return total / len(a)

def var(a):
  m = mean(a)
  b = [(a[i]-m) * (a[i]-m) for i in range (len(a))]
  return mean(b)

def corr(a,b):
  length = len(a)
  ma = mean(a)
  mb = mean(b)
  c = [(a[i]-ma)*(b[i]-mb) for i in range (length)]
  return mean(c)/((var(a)*var(b))**0.5)

def important_feat(data):
  feats = data.T
  good = []
  for i in range (len(feats)):
    (feat,output) = (feats[i], feats[-1])
    print(corr(feat, output))
    if (abs(corr(feat, output)) > 0.8):
      good.append(i)
  return good

def standardize(data, mean_of_all, var_of_all):
  feats = data.T
  for i in range (len(feats)-1):
    for j in range (len(feats[i])):
      feats[i][j] = (feats[i][j] - mean_of_all[i]) / (var_of_all[i]**0.5) + 2
    print(mean(feats[i]), var(feats[i]))
  return data

# Create your dataset
data = pd.read_csv("/content/train.csv")
data = data.values
mean_of_all = [mean(data.T[i]) for i in range (len(data.T))]
var_of_all = [var(data.T[i]) for i in range (len(data.T))]
print(mean_of_all)
data = standardize(data, mean_of_all, var_of_all)
train_data = np.transpose(np.array(np.float64(data)))
feats = [14]
feats = important_feat(data)
print(feats)

[24.51517145826116, 0.32932282646345823, 2.440335988915837, 9.962660200900599, 12.446917215102188, 31.00419120193976, 22.560443366816767, 1.4311915483200521, 0.26548320055420843, 73.28074125389678, 1.670782819535851, 162.68877727745064, 163.91340491860063, 1.8058538275025928, 11.582784897817804]
1.9999999999999993 1.0000000000000093
2.0000000000000138 1.0000000000000029
1.9999999999999862 0.9999999999999614
2.0000000000000138 0.9999999999999986
2.000000000000008 0.9999999999999931
1.9999999999999976 0.9999999999999915
1.9999999999999998 0.9999999999999861
2.00000000000001 0.9999999999999706
2.0000000000000795 0.9999999999998882
1.9999999999999993 1.0000000000000115
1.9999999999999947 0.999999999999985
1.9999999999999925 0.9999999999999981
1.9999999999999465 1.0000000000000004
2.000000000000003 0.9999999999999736
-0.17614746492930033
0.6591476679747424
0.22721914669686993
0.5542736873991053
0.5136501396100859
0.2339239437855961
0.8188682137702156
-0.10204740464347103
-0.0608012214487119

In [60]:
def valid(data, index, extreme):
  for i in range (8):
    for j in range (15):
      if (data[j,index+i] > extreme[j][0]):
        return False
      if (data[j,index+i] < extreme[j][1]):
        return False
  # COMPLETED: Try to filter out extreme values.
  #  ex: If PM2.5 > 100, then we don't use the data to train (return False), otherwise return True,
  return True

def get_extreme(data):
  extreme = []
  for i in range (len(data)):
    a = mean(data[i]) + 10*var(data[i])**0.5
    b = mean(data[i]) - 10*var(data[i])**0.5
    extreme.append([a,b])
  return extreme

def parse2train(data, feats):

  x = []
  y = []

  # Use data #0~#7 to predict #8 => Total data length should be decresased by 8.
  total_length = data.shape[1] - 8

  extreme_value = get_extreme(data)
  print(extreme_value)

  for i in range(total_length):
    x_tmp = data[feats, i:i+8] # Use data #0~#7 to predict #8, data #1~#8 to predict #9, etc.
    y_tmp = data[-1, i+8] # last column of (i+8)th row: PM2.5

    # Filter out extreme values to train.
    if valid(data,i,extreme_value):
      x.append(x_tmp.reshape(-1,))
      y.append(y_tmp)
  
  # x.shape: (n, 15, 8)
  # y.shape: (n, 1) 
  x = np.array(x)
  y = np.array(y)

  return x,y


train_x, train_y = parse2train(train_data, feats)
print("fweuifbifbewbew")
print(train_x.shape)
print("nfewiofbwebeowo")
print(train_y.shape)
#print(train_x[0,])


[[12.000000000000046, -8.000000000000046], [12.000000000000028, -8.0], [11.999999999999792, -7.999999999999821], [12.000000000000007, -7.999999999999979], [11.999999999999975, -7.999999999999958], [11.999999999999956, -7.99999999999996], [11.99999999999993, -7.999999999999931], [11.999999999999863, -7.999999999999842], [11.99999999999952, -7.9999999999993605], [12.000000000000057, -8.000000000000057], [11.99999999999992, -7.999999999999931], [11.999999999999982, -7.9999999999999964], [11.999999999999948, -8.000000000000055], [11.999999999999872, -7.999999999999865], [101.47649198145888, -78.31092218582327]]
fweuifbifbewbew
(5614, 16)
nfewiofbwebeowo
(5614,)


In [61]:
def add_squares(x):
  xx = x.copy()
  for i in range (len(x)):
    for j in range (len(x[i])):
      xx[i][j] *= xx[i][j]
  # print(x, x.shape)
  # print(xx, xx.shape)
  xxx = np.concatenate((x,xx), axis=1)
  # print(xxx, xxx.shape)
  return xxx

#train_x = add_squares(train_x)

In [62]:

from argparse import Namespace

# TODO: Tune the config to boost your performance. 
train_config = Namespace(
    batch_size = 512,
    lr = 0.08,
    lam = 0.001,
    epoch = 1000,
)

  # TODO: Implement 2-nd polynomial regression version for the report.
def minibatch(x, y, config):
  
    # Randomize the data in minibatch
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    y = y[index]
    
    # Initialization
    batch_size = config.batch_size
    lr = config.lr
    lam = config.lam
    epoch = config.epoch

    beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1)
    beta_2 = np.full(x[0].shape, 0.99).reshape(-1, 1)
    # Linear regression: only contains two parameters (w, b).
    w = np.full(x[0].shape, 0.1).reshape(-1, 1)
    bias = 0.1
    m_t = np.full(x[0].shape, 0).reshape(-1, 1)
    v_t = np.full(x[0].shape, 0).reshape(-1, 1)
    m_t_b = 0.0
    v_t_b = 0.0
    t = 0
    epsilon = 1e-8
    
    # Training loop
    for num in range(epoch):
        for b in range(int(x.shape[0]/batch_size)):
            t+=1
            x_batch = x[b*batch_size:(b+1)*batch_size]
            y_batch = y[b*batch_size:(b+1)*batch_size].reshape(-1,1)

            # Prediction of linear regression 
            pred = np.dot(x_batch,w) + bias
            # loss
            loss = y_batch - pred
            
            # Compute gradient
            g_t = np.dot(x_batch.transpose(),loss) * (-2) +  2 * lam * np.sum(w) 
            g_t_b = loss.sum(axis=0) * (-2)
            m_t = beta_1*m_t + (1-beta_1)*g_t 
            v_t = beta_2*v_t + (1-beta_2)*np.multiply(g_t, g_t)
            m_cap = m_t/(1-(beta_1**t))
            v_cap = v_t/(1-(beta_2**t))
            m_t_b = 0.9*m_t_b + (1-0.9)*g_t_b
            v_t_b = 0.99*v_t_b + (1-0.99)*(g_t_b*g_t_b) 
            m_cap_b = m_t_b/(1-(0.9**t))
            v_cap_b = v_t_b/(1-(0.99**t))
            w_0 = np.copy(w)
            
            # Update weight & bias
            w -= ((lr*m_cap)/(np.sqrt(v_cap)+epsilon)).reshape(-1, 1)
            bias -= (lr*m_cap_b)/(math.sqrt(v_cap_b)+epsilon)
        # print(num, temp)
    return w, bias

def testing(w, bias, test_x, test_y):
  a = []
  for i in range (test_y.shape[0]):
    prediction = (np.dot(np.reshape(w,-1),test_x[i]) + bias)[0]
    a.append((test_y[i] - prediction) * (test_y[i] - prediction))
  return mean(a)**0.5



# Train your regression model

group = [random.randint(1,10) for i in range(train_x.shape[0])]
total = 0
for i in range (10):
  choose = [group[j] == i+1 for j in range(train_x.shape[0])]
  notchoose = [group[j] != i+1 for j in range(train_x.shape[0])]
  test_x = train_x[choose]
  test_y = train_y[choose]
  train_x_new = train_x[notchoose]
  train_y_new = train_y[notchoose]
  w, bias = minibatch(train_x_new, train_y_new, train_config)
  # print(w, bias, w.shape)
  rmse = testing(w, bias, test_x, test_y)
  print(rmse)
  total += rmse
print("total = ", total/10)



w, bias = minibatch(train_x, train_y, train_config)
print(w, bias, w.shape)


3.507766846954232
3.33374439074881
3.4968326417629263
3.75141380987473
3.965045514000591
3.584588296163968
3.314604243411271
4.174020061645189
3.881964098134533
3.211636266886514
total =  3.6221616169582767
[[-3.69025812e-01]
 [-1.80681694e-01]
 [-2.13692644e-01]
 [-1.77231707e-01]
 [-1.46316172e-01]
 [-3.63535966e-01]
 [-4.97111925e-01]
 [ 2.25853236e+00]
 [ 3.85964520e-02]
 [ 3.45312633e-02]
 [ 3.43725742e-03]
 [ 3.04075023e-02]
 [-1.62772659e-03]
 [ 3.98606364e-02]
 [ 6.63634638e-02]
 [ 6.52880567e-01]] [0.53636856] (16, 1)


In [63]:
def parse2test(data, feats):
  x = []
  for i in range(90):
    x_tmp = data[feats,8*i: 8*i+8]
    x.append(x_tmp.reshape(-1,))

  # x.shape: (n, 15, 8)
  x = np.array(x)
  return x

def switching(data):
  for i in range(data.shape[0]):
    temp = data[i,7]
    data[i,7] = data[i,14]
    data[i,14] = temp
  return data

data = pd.read_csv('test.csv')
data = data.values
data = switching(data)
data = standardize(data, mean_of_all, var_of_all)

test_data = np.transpose(np.array(np.float64(data)))
test_x = parse2test(test_data, feats)
#test_x = add_squares(test_x)

with open('my_sol.csv', 'w', newline='') as csvf:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvf)
    writer.writerow(['Id','Predicted'])

    print(test_x.shape) 
    for i in range(int(test_x.shape[0])):
      # Prediction of linear regression 
      prediction = (np.dot(np.reshape(w,-1),test_x[i]) + bias)[0]
      writer.writerow([i, prediction] )
      print(prediction)
from google.colab import files
files.download("my_sol.csv")

1.521273730702839 0.52287119097821
1.3938062100970234 0.5011070056686319
1.6466886241691507 0.676276502796678
1.548039476961266 0.4452643868508214
1.522563739757636 0.524651764096304
1.8556499223928635 0.43159501977243464
1.625824670657835 0.36030859702100976
1.9917472395422453 0.8748021262052255
1.9510666364806357 0.15105717148684603
2.627494123741949 0.6931760068062793
1.2700923737193046 0.12971386244868402
1.541068261539133 0.332216829076591
1.5788455316712149 0.4557768671845273
1.88157218577735 0.8560397925612523
(90, 16)
7.8073890860199375
2.0667021545869964
11.80289508198277
13.406791176640604
6.604803886697368
6.618370686247417
7.992839423027207
10.044012447798055
12.837365421810132
5.675631539044523
6.445292002710979
8.523402990062948
9.931570285494308
3.5616358521753484
4.7588947445250795
6.016938621426575
2.3395996591553327
6.240687617252138
8.192396282050785
3.063302423139619
2.3312786638169056
8.031827115800509
4.584690675808199
6.889940689431179
2.9765083654547904
8.380705

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>